<h1>Bayesian PAC (One Subject)</h1>

The <strong>Phase-Amplitude Coupling (PAC) analysis</strong> is a technique used in neuroscience to study the interaction between different frequency bands in brain signals (in our case EEG signals).

The <strong>Bayesian PAC</strong> approach that we are trying to develop is an extension that uses Bayesian theory <strong>to provide a probabilistic interpretation of neural connections</strong>.

<h2>Initialization</h2>

<h3>Libraries</h3>

In [1]:
import matplotlib.pyplot as plt
import mne
import numpy as np
import pandas as pd
import seaborn as sns
import tensorpac

from scipy.signal import hilbert
from scipy.stats import gaussian_kde, norm

<h3>Global Vars</h3>

In [2]:
G_nChannels = 31     # We have 32 channels though one (Cz) is used as reference for offset and normalization purposes.
G_Fs = 500           # fSampling (Hz).
G_Alpha = 0.05       # Reference p-Value to evaluate the z-Score threshold.
G_nSurrogates = 200  # Number of surrogates to evaluate.
G_nFragments = 25    # Number of fragments (should be divisor of the total lenght of EEG signals).
G_nBins = 40         # Number of bins for the histogram of significant PAC values.
G_EEG_labels = ['Fp1','Fp2','F7','F3','Fz','F4','F8','FC5','FC1','FC2','FC6','T7','C3','C4','T8','TP9','CP5','CP1','CP2','CP6','TP10','P7','P3','PZ','P4','P8','PO9','O1','OZ','O2','PO10']

<h3>Reading the EEG signal from the subject of interest</h3>

In [3]:
# Reading the EEG signal we want to evaluate
fName = 'your_eeg_signal_sample.npy'
fName = '043_C7_8b.npy'
npy_load = np.load('DDBB/'+fName,mmap_mode='r',allow_pickle=True) # Replace 'DDBB/' with the path to your DDBB.
npy_data = np.nan_to_num(npy_load.copy())

<h2>Auxiliary Functions</h2>

<h3>Filtering</h3>

In [4]:
def plot_filtered_eeg(signal, time_range, labels, n_channels, title):
    """
    Plot filtered EEG signals.
    
    Parameters:
    signal (numpy.ndarray): EEG signal data.
    time_range (list): Time range to plot.
    labels (list): EEG channel labels.
    n_channels (int): Number of EEG channels.
    title (str): Title of the plot.
    """
    t = np.linspace(time_range[0], time_range[1], time_range[1] - time_range[0] + 1, dtype=int)
    fig, ax = plt.subplots(nrows=4, ncols=8, figsize=(15, 10))
    my_colors = sns.color_palette("rocket", n_colors=n_channels)

    for i in range(n_channels):
        ax.flatten()[i].plot(t, signal[i, t], alpha=0.75, linewidth=0.75, color=my_colors[i], label=labels[i])
        ax.flatten()[i].legend(loc='upper right', fontsize='small')
    
    fig.suptitle(title)
    plt.tight_layout(rect=[0, 0.03, 1, 0.95])

    plt.savefig('Figures/temp_signal_'+title.replace(" ", "")+'.eps', format='eps', bbox_inches='tight')
    plt.savefig('Figures/temp_signal_'+title.replace(" ", "")+'.png', format='png', bbox_inches='tight')

    plt.show()

<h3>Visualization</h3>

In [5]:
def plot_pac_heatmap(significant_pac_values, labels, is_fragmented=False):
    """
    Plot a heatmap of significant PAC values.
    
    Parameters:
    significant_pac_values (numpy.ndarray): Significant PAC values between pairs of electrodes.
    labels (list): EEG channel labels.
    is_fragmented (bool): If True, significant_pac_values is assumed to be averaged values from fragments.
    """
    title = 'Significant PAC Values Heatmap (averaged results of the fragments)' if is_fragmented else 'Significant PAC Values Heatmap (Complete Signal)'
    
    if is_fragmented:
        averaged_pac_values = np.mean(significant_pac_values, axis=2)
    else:
        averaged_pac_values = significant_pac_values
    
    pac_df = pd.DataFrame(averaged_pac_values * 1e5, index=labels, columns=labels)  # Scale PAC values
    plt.figure(figsize=(12, 10))
    sns.heatmap(pac_df, annot=True, cmap="rocket", linewidths=.5, fmt=".1f", annot_kws={"size": 6})
    cbar = plt.gcf().axes[-1]
    cbar.set_ylabel('PAC Value (x10⁻⁵)', rotation=90, fontsize=12)
    cbar.yaxis.set_label_position('right')
    plt.title(title)

     # Set labels
    plt.xlabel('Sources', fontsize=12)
    plt.ylabel('Destinations', fontsize=12)

    if is_fragmented:
        trail_string = 'fragmented'
    else:
        trail_string = 'full'
    plt.savefig('Figures/significant_pac_heatmap_'+trail_string+'.eps', format='eps', bbox_inches='tight')
    plt.savefig('Figures/significant_pac_heatmap_'+trail_string+'.png', format='png', bbox_inches='tight')

    plt.show()



def plot_bayesian_pac_heatmap(P_i_given_x, labels, save_str='default'):
    """
    Plot a heatmap of the Bayesian PAC values P(i|x) as percentages.
    
    Parameters:
    P_i_given_x (numpy.ndarray): The Bayesian PAC values matrix of shape (destinations, sources).
    labels (list): List of labels for the source and destination electrodes.
    save_str (str): String used for saving the figures.
    """
    # Convert the matrix to percentages
    P_i_given_x_percentage = P_i_given_x * 100
    
    # Calculate the maximum value and round up to nearest multiple of 2
    max_value = np.ceil(np.max(P_i_given_x_percentage) / 2) * 2
    
    # Create tick sequence from 0 to max_value
    ticks = np.arange(0, max_value + 2, 2)  # +2 to include the max value
    
    # Set vmax to the maximum tick value for consistent scale
    vmax = ticks[-1]
    
    # Convert the matrix to a DataFrame for easier plotting with seaborn
    P_i_given_x_df = pd.DataFrame(P_i_given_x_percentage, index=labels, columns=labels)

    # Create the figure with specific size
    plt.figure(figsize=(12, 10))
    
    # Heatmap without annotations
    # ax = sns.heatmap(P_i_given_x_df, 
    #                  annot=False, 
    #                  cmap="rocket", 
    #                  linewidths=.5,
    #                  vmin=0,
    #                  vmax=vmax)
    
    # Heatmap with annotations
    ax = sns.heatmap(P_i_given_x_df,
                     annot=True,
                     cmap="rocket",
                     linewidths=.5,
                     vmin=0,
                     vmax=vmax,
                     fmt=".1f",
                     annot_kws={"size": 6})

    # Set labels
    plt.xlabel('Sources', fontsize=12)
    plt.ylabel('Destinations', fontsize=12)
    
    # Modify colorbar
    colorbar = ax.collections[0].colorbar
    colorbar.set_label('Bayesian PAC Values P(i|x)', fontsize=12)
    
    # Set percentage ticks on colorbar
    colorbar.set_ticks(ticks)
    colorbar.set_ticklabels([f'{tick}%' for tick in ticks])
    
    # Save the plot
    plt.savefig(f'Figures/bayesian_pac_heatmap_{save_str}.eps', format='eps', bbox_inches='tight')
    plt.savefig(f'Figures/bayesian_pac_heatmap_{save_str}.png', format='png', bbox_inches='tight')
    
    # Show the plot
    plt.show()

<h2>Main Pipeline</h2>

<h3>Step 1: Filtering the EEG signal on interest bands</h3>

In [ ]:
# Example filtering Theta and Gamma bands
data_theta = mne.filter.filter_data(npy_data, G_Fs, 4, 8)
data_gamma = mne.filter.filter_data(npy_data, G_Fs, 30, 80)

# Plot filtered EEG signals (Optional. This is an example showing a segment of the EEG signal and its filtered Theta and Gamma bands)
plot_filtered_eeg(npy_data, [25000, 25400], G_EEG_labels, G_nChannels, "Not filtered")
plot_filtered_eeg(data_theta, [25000, 25400], G_EEG_labels, G_nChannels, "Theta band")
plot_filtered_eeg(data_gamma, [25000, 25400], G_EEG_labels, G_nChannels, "Gamma band")

<h3>Step 2: Hilbert Transform</h3>

In [7]:
# Applying Hilbert Transform to obtain the phase (ip) and the amplitude (ia) of Theta/Gamma bands
ip_theta = np.angle(hilbert(data_theta, axis=1)) # Results present a size: [nChannels,Time]
ia_gamma = np.abs(hilbert(data_gamma, axis=1))   # Results present a size: [nChannels,Time]

# Reshape to add the n_epochs dimension
ip_theta = ip_theta[:, np.newaxis, :]
ia_gamma = ia_gamma[:, np.newaxis, :]

<h3>Step 3: Segmentation</h3>

In [8]:
L_fLength = int(np.shape(npy_data)[1]/G_nFragments) # Fragment length

# Create arrays for fragments
ip_theta_fragments = np.zeros((G_nChannels, G_nFragments, L_fLength))
ia_gamma_fragments = np.zeros((G_nChannels, G_nFragments, L_fLength))

# Filling the matrices with the fragments
for i in range(G_nFragments):
    ip_theta_fragments[:, i, :] = ip_theta[:, 0, i * L_fLength: (i + 1) * L_fLength]
    ia_gamma_fragments[:, i, :] = ia_gamma[:, 0, i * L_fLength: (i + 1) * L_fLength]

<h3>Step 4: Computing PAC values using TensorPAC with surrogates</h3>

The PAC analyzes the relationship between the phase of a low-frequency oscillation (e.g., theta: 4-8 Hz) and the amplitude of a high-frequency oscillation (e.g., gamma: 30-80 Hz). The concept is that the phase of the low frequency signal can modulate the amplitude of the high frequency signal.

In [ ]:
# Computing the threshold for significance tests using z-scores (Bonferroni correction)
L_alpha_Bonferroni = G_Alpha/G_nFragments
G_zThreshold_Bonferroni = norm.ppf(1 - L_alpha_Bonferroni)
print(f"z-Score threshold for the fragmented signal ({G_nFragments} fragments) [Bonferroni-Correction]: {G_zThreshold_Bonferroni}")

# Initializing arrays to store results (fragments)
pac_values_fragments = np.zeros((G_nChannels, G_nChannels, G_nFragments))
surrogate_pac_values_fragments = np.zeros((G_nChannels, G_nChannels, G_nSurrogates, G_nFragments))
p_values_fragments = np.zeros((G_nChannels, G_nChannels, G_nFragments))

# Initialize PAC object with surrogates
p = tensorpac.Pac(idpac=(2, 2, 0), f_pha=(4, 8), f_amp=(30, 80), dcomplex='wavelet', width=7)

# Calculate PAC by fragments
for frag in range(G_nFragments):
    for i in range(G_nChannels):
            for j in range(G_nChannels):
                pac = p.fit(ip_theta_fragments[i, frag].reshape(1, 1, -1), 
                            ia_gamma_fragments[j, frag].reshape(1, 1, -1), 
                            n_perm=200, 
                            random_state=1234)
                pac_values_fragments[i, j, frag] = p.pac[0][0][0]
                surrogate_pac_values_fragments[i, j, :, frag] = p.surrogates.reshape(G_nSurrogates)
                p_values_fragments[i, j, frag] = p.pvalues[0][0]

# Calculate z-scores and determine significant PAC values per fragments
z_scores_fragments = np.zeros((G_nChannels, G_nChannels, G_nFragments))
significant_pac_values_fragments = np.zeros((G_nChannels, G_nChannels, G_nFragments))

for frag in range(G_nFragments):
    for i in range(G_nChannels):
        for j in range(G_nChannels):
            real_pac = pac_values_fragments[i, j, frag]
            surrogate_mean = np.mean(surrogate_pac_values_fragments[i, j, :, frag])
            surrogate_std = np.std(surrogate_pac_values_fragments[i, j, :, frag])
            if surrogate_std == 0:
                z_scores_fragments[i, j, frag] = 0  # Avoid division by zero
            else:
                z_scores_fragments[i, j, frag] = (real_pac - surrogate_mean) / surrogate_std

            if abs(z_scores_fragments[i, j, frag]) > G_zThreshold_Bonferroni:
                significant_pac_values_fragments[i, j, frag] = pac_values_fragments[i, j, frag]

# Check dimensions
print(np.shape(z_scores_fragments))                 # Should be (G_nChannels, G_nChannels, G_nFragments)
print(np.shape(significant_pac_values_fragments))   # Should be (G_nChannels, G_nChannels, G_nFragments)

# Significant PACs of fragments
plot_pac_heatmap(significant_pac_values_fragments, G_EEG_labels, is_fragmented=True)

<h3>Step 5: Bayes' Theorem</h3>

The <strong>Bayesian PAC</strong> introduces a probabilistic perspective to evaluate the connections between different nodes in the brain. It uses Bayes' Theorem to calculate the probability that a target node $i$ is activated, conditional on a source node $x$ being exerting an influence.

Our objective is to determine the term $P(i∣x)$ which represents the probability that target node $i$ is activated given that source node $x$ is exerting an influence.

$$P(i|x) = \frac{P(x|i) \cdot P(i)}{P(x)} = \frac{P(x|i)·P(i)}{\sum\limits_i P(x|i) \cdot P(i)}$$

where...

* $P(i)$: Probability (a priori) of activation at electrode $i$. This means, $P(i)$ represents the probability of having the destination node $i$ activated regardless of any source node $x$. Using other words, this term represents the proportion of times that the target node $i$ shows significant activation across all fragments and node pairs.

$$P(i) = \frac{\# \text{significant PACs at electrode } i \text{ with origin any other electrode}}{\# \text{significant PACs}}$$

* $P(x)$: Probability of observing an influence of source node $x$ in general.

* $P(x∣i)$: indicates the probability of observing an influence of source node $x$ given that target node $i$ is activated. We can determine its value by using the probability density function (obtained using KDE) for electrode $i$.

<h4>Histogram of significant PAC values</h4>

In [ ]:
# Flatten the significant PAC values matrix for fragments and remove zeros
significant_pac_values_flat_fragments = significant_pac_values_fragments.flatten()
significant_pac_values_flat_fragments = significant_pac_values_flat_fragments[significant_pac_values_flat_fragments > 0] * 1e5 # Scale to avoid problems with small values

# Fit a KDE to the data
kde_fragments = gaussian_kde(np.abs(significant_pac_values_flat_fragments)) # We make use only of absolute values

# Fit a Gaussian distribution to the data
mu_fragments, std_fragments = norm.fit(np.abs(significant_pac_values_flat_fragments)) # We make use only of absolute values

# Plot the histogram with frequencies
fig, ax1 = plt.subplots(figsize=(10, 6))
n, bins, patches = ax1.hist(significant_pac_values_flat_fragments, bins=G_nBins, color='skyblue', edgecolor='black', rwidth=0.8)
# Create a secondary y-axis to plot the KDE and Gaussian PDF
ax2 = ax1.twinx()
# Plot the KDE
xmin, xmax = ax1.get_xlim()
x = np.linspace(xmin, xmax, 100)
p_kde_fragments = kde_fragments(x)
ax2.plot(x, p_kde_fragments, 'r', linewidth=2, label='KDE')
# Plot the Gaussian PDF
p_gauss_fragments = norm.pdf(x, mu_fragments, std_fragments)
ax2.plot(x, p_gauss_fragments, 'm', linewidth=2, label='Gaussian PDF')
# Set titles and labels
ax1.set_title("Histogram of Significant PAC Values (Fragments)")
ax1.set_xlabel("PAC Value (x10⁻⁵)")
ax1.set_ylabel("Frequency")
ax2.set_ylabel("Probability Density")
ax1.grid(True)
ax2.legend()
plt.savefig('Figures/histogram_and_KDE.eps', format='eps', bbox_inches='tight')
plt.savefig('Figures/histogram_and_KDE.png', format='png', bbox_inches='tight')
plt.show()

<h4>Computing P(i)</h4>

In [11]:
# Calculate P(i) for each destination node
significant_pac_counts_by_destination = np.zeros(G_nChannels)
for frag in range(G_nFragments):
    for i in range(G_nChannels):
        for j in range(G_nChannels):
            if significant_pac_values_fragments[i, j, frag] > 0:
                significant_pac_counts_by_destination[i] += 1
total_significant_pacs = np.sum(significant_pac_counts_by_destination)
P_i = significant_pac_counts_by_destination / total_significant_pacs

<h4>Computing P(x|i)</h4>

In [12]:
# Initialize P(x|i) and the counters necessary to obtain significant values.
P_x_given_i = np.zeros((G_nChannels, G_nChannels, G_nFragments))
P_x_given_i_significant_counts = np.zeros((G_nChannels, G_nChannels))

# Evaluate KDE for each significant PAC value
for frag in range(G_nFragments):
    for i in range(G_nChannels):
        for j in range(G_nChannels):
            if significant_pac_values_fragments[i, j, frag] > 0:
                P_x_given_i[i, j, frag] = kde_fragments(significant_pac_values_fragments[i, j, frag] * 1e5).item()
            else:
                P_x_given_i[i, j, frag] = 0

# Normalize P(x|i) by fragment and count significant values
for frag in range(G_nFragments):
    P_x_given_i_sum = np.sum(P_x_given_i[:, :, frag], axis=0)
    P_x_given_i[:, :, frag] = np.divide(P_x_given_i[:, :, frag], P_x_given_i_sum, 
                                        out=np.zeros_like(P_x_given_i[:, :, frag]), 
                                        where=P_x_given_i_sum != 0)
    P_x_given_i_significant_counts += (P_x_given_i[:, :, frag] > 0).astype(int)

# Calculate the proportion of significant fragments for P(x|i)
P_x_given_i_mean = P_x_given_i_significant_counts / G_nFragments

<h4>Computing P(x)</h4>

In [13]:
# Calculate P(x) for each fragment
P_x = np.zeros((G_nChannels, G_nFragments))
for frag in range(G_nFragments):
    P_x[:, frag] = np.dot(P_i, P_x_given_i[:, :, frag])

<h4>Applying Bayes' Theorem</h4>

In [14]:
# Calculate P(i|x) for each fragment using Bayes' Theorem and count the significant values
P_i_given_x_fragments = np.zeros((G_nChannels, G_nChannels, G_nFragments))
P_i_given_x_significant_counts = np.zeros((G_nChannels, G_nChannels))

for frag in range(G_nFragments):
    P_i_given_x_fragments[:, :, frag] = (P_x_given_i[:, :, frag].T * P_i).T
    P_i_given_x_fragments[:, :, frag] = np.divide(P_i_given_x_fragments[:, :, frag], P_x[:, frag], 
                                                  out=np.zeros_like(P_i_given_x_fragments[:, :, frag]), 
                                                  where=P_x[:, frag] != 0)
    P_i_given_x_significant_counts += (P_i_given_x_fragments[:, :, frag] > 0).astype(int)

# Calculate the proportion of significant fragments
P_i_given_x_mean = P_i_given_x_significant_counts / G_nFragments

<h4>Results</h4>

In [ ]:
# Represent the results using the display functions
plot_bayesian_pac_heatmap(P_i_given_x_mean, G_EEG_labels, 'fragmented-signal-aggregated-values')

# (Optional) Apply a threshold to filter out the most robust connections
P_i_given_x_thresholded = np.where(P_i_given_x_mean >= 0.1, P_i_given_x_mean, 0) # Only connections with proabilities >= 10% will be shown.
plot_bayesian_pac_heatmap(P_i_given_x_thresholded, G_EEG_labels, 'fragmented-signal-aggregated-values-threshold')